In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
#import plotly.express as px

#Tiesiog daugiau biblioteku instaliuojama.

In [2]:
#prisijungimas prie mySQL, ir kursoriaus sukūrimas
DB = cnt.connect(
host = '88.223.121.130',
user='user',
password='slaptazodis',
port=3306,
database='works'
)
C=DB.cursor()

In [17]:
#Turite nusiskaityti iš SQL DB lentelės autopliuslt į Pandas DataFrame gamintoją,
# kainą, pagaminimo datą, ridą, pavarų tipą, variklio informaciją. 
# SQL užklausoje turite atmesti tuos įrašus, kur rida nenurodyta, 
# taip pat palikite tik nesikartojančius įrašus (atranka pagal ID stuleplį - 
# jei ID sutampa - reiškia, jog skelbimas dubliuojasi)
sql="""with T1
as (select *, row_number() over (partition by id) as rc from autopliuslt)
select gamintojas, rida, price as kaina,
pagaminimo_data as data,
pavaros, variklis
from T1
where rc = 1 and rida != 'Nenurodyta' order by gamintojas"""
df = pd.read_sql_query(sql, con=DB)

df['R'] = df['rida'].apply(lambda s: float(s.replace(' ','').replace('km','')))
df['K'] = df['kaina'].apply(lambda s: float(s.replace(' ','').replace('km','')))
df['amžius'] = df['data'].apply(lambda s: 2024-int(s[:4]))

df.head()

,gamintojas,rida,kaina,data,pavaros,variklis,R,K,amžius
0,Alfa Romeo,305 000 km,2 100,2007-01,Mechaninė,"1910 cm³, 150 AG (110kW)",305000.0,2100.0,17
1,Alfa Romeo,308 615 km,4 900,2006-04,Mechaninė,"2387 cm³, 200 AG (147kW)",308615.0,4900.0,18
2,Aston Martin,45 000 km,125 000,2015-10,Automatinė,"5935 cm³, 517 AG (380kW)",45000.0,125000.0,9
3,Audi,112 000 km,25 000,2016-06,Automatinė,"3000 cm³, 333 AG (245kW)",112000.0,25000.0,8
4,Audi,450 000 km,7 800,2001-12,Automatinė,"4200 cm³, 360 AG (265kW)",450000.0,7800.0,23


In [19]:
#sql užklausa per pandas sql_query
#sql užklausa per pandas sql_query. Pridetas stulpelis su ridos intervalais 5000
sql="""with T1
as (select *, row_number() over (partition by id) as rc from autopliuslt)
select gamintojas, rida, price as kaina,
pagaminimo_data as data,
pavaros, variklis
from T1
where rc = 1 and rida != 'Nenurodyta' order by gamintojas"""
df = pd.read_sql_query(sql, con=DB)

df['R'] = df['rida'].apply(lambda s: float(s.replace(' ','').replace('km','')))
df['K'] = df['kaina'].apply(lambda s: float(s.replace(' ','').replace('km','')))
df['amžius'] = df['data'].apply(lambda s: 2024-int(s[:4]))
df['ridos_intervalas'] = np.ceil(df['R'] / 5000) * 5000
# galima ir taip:df['ridos_intervalas'] = df['R'].apply(lambda r:np.ceil(r / 5000) * 5000) 
df.head()


,gamintojas,rida,kaina,data,pavaros,variklis,R,K,amžius,ridos_intervalas
1681,Volvo,510 000 km,15 900,2017-04,Mechaninė,"2000 cm³, 150 AG (110kW)",510000.0,15900.0,7,510000.0
1682,Volvo,96 000 km,18 250,2016,Automatinė,"1969 cm³, 245 AG (180kW)",96000.0,18250.0,8,100000.0
1683,Volvo,373 000 km,1 650,2005-01,Mechaninė,"2500 cm³, 220 AG (162kW)",373000.0,1650.0,19,375000.0
1684,Volvo,270 000 km,17 900,2015-04,Automatinė,"2400 cm³, 230 AG (169kW)",270000.0,17900.0,9,270000.0
1685,Volvo,250 000 km,5 950,2007-07,Automatinė,"2401 cm³, 185 AG (136kW)",250000.0,5950.0,17,250000.0


In [ ]:
#+ prie to kos buvo
dfg=df
fig, axis = plt.subplots(figsize=(8,4.5))
#plotting code:

axis.scatter(dfg['R5000'], dfg['K'])

#axis.legend(loc='best')
#axis.ticklabel_format(style='plain')
fig.tight_layout()
plt.show()